In [32]:
from pandas_datareader import data, wb
import pandas as pd
import numpy as np
import datetime
import csv
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor

In [33]:
def predict_clossing_price(start_year, start_month, start_day, code):
    
    start = datetime.datetime(start_year,start_month,start_day)
    end = datetime.date.today()
    stockData = data.DataReader(code , 'yahoo',start,end)

    def get_rsi(sdata,m,mem):
        neg = 0
        pos = 0
        RS = 0
        RSI = 100

        upcloses = 0
        downcloses = 0

        n = m
        k = m-1

        for p in range(mem):
           diff = sdata[n,3] - sdata[k,3]
           if (diff>=0):
            upcloses = upcloses + diff
            pos = pos+1
           else:
            downcloses = downcloses + diff
            neg = neg+1

           n = n-1
           k = k-1

        downcloses = -downcloses
        if(neg == 0):
            return 100
        else:
            RS = (upcloses*neg)/(downcloses*pos)


        RSI = 100 - (100/(1+RS))

        return RSI


    def get_mfi(sdata,m,mem):
        neg = 0
        pos = 0
        MFR = 0
        MFI = 100

        pmflow = 0
        nmflow = 0

        n = m
        k = m-1

        for p in range(mem):

           typ_pricec = (sdata[n,0] + sdata[n,1] + sdata[n,3])/3
           typ_pricep = (sdata[k,0] + sdata[k,1] + sdata[k,3])/3

          # print(typ_price,sdata[n,0],sdata[n,1],sdata[n,3])

           if (typ_pricec>=typ_pricep):
             pmflow = pmflow + ((sdata[n,4])*(typ_pricec))
             pos = pos+1
           else:
             nmflow = nmflow + ((sdata[n,4])*(typ_pricec))
             neg = neg+1

           n = n-1
           k = k-1

        if(neg == 0):
            return 100
        else:
            MFR = pmflow/nmflow

        MFI = 100 - (100/(1+MFR))

        return MFI

    def get_ema(sdata,m,mem,EMAp):

       EMA = sdata[m,3]*(2/(1+mem)) + (1-(2/(1+mem)))*EMAp 
       #print("EMA",EMA)

       return EMA

    def get_so(sdata,m,mem):

       SO = ((sdata[m,3]-sdata[m,1])/(sdata[m,0]-sdata[m,1]))*100
       #print("SO",SO)
       return SO


    memory = 14
    sdat2 = stockData.reset_index()
    del sdat2["Date"]
    del sdat2["Adj Close"]
    sdat3 = np.array(sdat2,dtype=np.float32)

    #print(sdat3[0])

    df1 = pd.DataFrame(columns=['Open','High','Low','Close','Volume','RSI','MFI','EMA','SO','CloseNext'])

    df2 = pd.DataFrame(columns=['Close','RSI','MFI','EMA','SO','CloseNext'])

    arr = np.array(df1.values)

    #print("Printing j\n")

    EMAp = 0
    acc = 0

    for i in range(memory):
        acc = acc + sdat3[i,3]

    EMAp = acc / memory    

    #for i in range(len(sdat3) -memory):
    for i in range(len(sdat3)-1 -memory):
        j = i + memory

        RSI = get_rsi(sdat3,j,memory)
        #print("RSI:",RSI)

        MFI = get_mfi(sdat3,j,memory)
        #print("MFI:",MFI)

        EMA = get_ema(sdat3,j,memory,EMAp)
        EMAp = EMA

        SO = get_so(sdat3,j,memory)

        N_close = sdat3[j+1,3]

        rec1 = [sdat3[j,2],sdat3[j,0],sdat3[j,1],sdat3[j,3],sdat3[j,4],RSI,MFI,EMA,SO,N_close]
        rec2 = [sdat3[j,3],RSI,MFI,EMA,SO,N_close]

        if(sdat3[j,4]!=0):
            d1 = {"Open":sdat3[j,2],"High":sdat3[j,0],"Low":sdat3[j,1],"Close":sdat3[j,3],"Volume":sdat3[j,4],"RSI":RSI,"MFI":MFI,"EMA":EMA,"SO":SO,"CloseNext":N_close}
            df1.loc[i] = rec1
            df2.loc[i] = rec2

    #ANN
    dataset = df1
    y = pd.DataFrame(dataset['CloseNext'])
    X = dataset.drop(['CloseNext'], axis = 1)

    X = np.array(X)
    y = np.array(y)
    #X = X[1700:2030,:]
    #y = y[1700:2030,:]
    y = y.flatten()

    #Feature scaling
    scaled = StandardScaler()
    scaled.fit(X)
    X = scaled.transform(X)

    #Train Test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

    ## ANN implementation
    # define base model of our neural network for regression taks
    def endgame():
        # Adding the neurons in various layers
        model = Sequential()
        model.add(Dense(9, input_dim=9, kernel_initializer='normal', activation='relu'))
        model.add(Dense(15, kernel_initializer='normal', activation='relu'))
        model.add(Dense(8, kernel_initializer='normal', activation='relu'))
        model.add(Dense(1, kernel_initializer='normal'))
        # Compile model for our use in KerasRegressor
        model.compile(loss='mean_squared_error', optimizer='adam', metrics = ['mape'])
        return model

    ann_regression = KerasRegressor(build_fn = endgame, epochs=100, batch_size=5, verbose=1)

    ann_regression.fit(X_train,y_train)

    ann_predict = ann_regression.predict(X_test)

    error = mean_absolute_error(ann_predict,y_test)
    per_err = (error/np.mean(y_test)) * 100
    print('The mean absolute error is {} and percentage error is {}.'.format(error,per_err))
    
    return ann_regression, scaled, df1

In [34]:
#parameter
start_year = 2011
start_month = 1
start_day = 1
code = "RELIANCE.NS"

ann_regression, scaled, df1  = predict_clossing_price(start_year, start_month, start_day, code)

<ipython-input-33-fed9038c8ae3>:90: RuntimeWarning: invalid value encountered in float_scalars
  SO = ((sdata[m,3]-sdata[m,1])/(sdata[m,0]-sdata[m,1]))*100


Epoch 1/100
349/349 [==============================] - 1s 2ms/step - loss: 832413.8423 - mape: 98.9987
Epoch 2/100
349/349 [==============================] - 1s 1ms/step - loss: 250155.4067 - mape: 45.3383
Epoch 3/100
349/349 [==============================] - 1s 2ms/step - loss: 43139.4495 - mape: 21.6647
Epoch 4/100
349/349 [==============================] - 0s 1ms/step - loss: 28393.3504 - mape: 17.9030
Epoch 5/100
349/349 [==============================] - 1s 1ms/step - loss: 20132.7643 - mape: 15.2533
Epoch 6/100
349/349 [==============================] - 0s 1ms/step - loss: 11430.6332 - mape: 11.7479
Epoch 7/100
349/349 [==============================] - 0s 1ms/step - loss: 4515.4443 - mape: 8.3131
Epoch 8/100
349/349 [==============================] - 0s 1ms/step - loss: 1693.4615 - mape: 5.4030
Epoch 9/100
349/349 [==============================] - 0s 1ms/step - loss: 793.5651 - mape: 3.7134
Epoch 10/100
349/349 [==============================] - 1s 2ms/step - loss: 684.9593 - 

349/349 [==============================] - 0s 1ms/step - loss: 440.4510 - mape: 1.6041
Epoch 83/100
349/349 [==============================] - 0s 1ms/step - loss: 361.6035 - mape: 1.4526
Epoch 84/100
349/349 [==============================] - 1s 1ms/step - loss: 440.8601 - mape: 1.5388
Epoch 85/100
349/349 [==============================] - 1s 1ms/step - loss: 422.7699 - mape: 1.5462
Epoch 86/100
349/349 [==============================] - 1s 1ms/step - loss: 379.0153 - mape: 1.4887
Epoch 87/100
349/349 [==============================] - 0s 1ms/step - loss: 470.6535 - mape: 1.6060
Epoch 88/100
349/349 [==============================] - 0s 1ms/step - loss: 445.6549 - mape: 1.5826
Epoch 89/100
349/349 [==============================] - 1s 1ms/step - loss: 394.3830 - mape: 1.5249
Epoch 90/100
349/349 [==============================] - 1s 1ms/step - loss: 406.5718 - mape: 1.4924
Epoch 91/100
349/349 [==============================] - 0s 1ms/step - loss: 460.5757 - mape: 1.6292
Epoch 92/100


In [50]:
text = "2207	2215.100098	2170.25	2181.949951	5316182	48.38902871	55.85965509	2133.179793	26.08678937"
X = [[float(i) for i in text.split()]]
ann_regression.predict(scaled.transform(np.array(X)))

1/1 [==============================] - 0s 37ms/step


array(2193.2798, dtype=float32)

In [36]:
df1

,Open,High,Low,Close,Volume,RSI,MFI,EMA,SO,CloseNext
0,482.922363,491.590210,481.931763,488.766968,9012481.0,54.901661,37.582003,506.609041,70.769191,481.238342
1,492.878021,493.274261,477.523560,481.238342,10265008.0,51.823167,30.253539,503.226281,23.584871,474.749847
2,483.962524,487.850677,473.635406,474.749847,9380240.0,49.922295,21.880954,499.429423,7.839745,466.924042
3,478.464630,479.158051,465.586700,466.924042,12970523.0,53.557356,13.980960,495.095373,9.854151,452.213470
4,468.063232,468.509003,446.765106,452.213470,15624401.0,52.254094,13.454348,489.377786,25.056982,455.333893
...,...,...,...,...,...,...,...,...,...,...
2490,2180.000000,2189.949951,2157.699951,2175.850098,9892597.0,49.144867,56.481724,2092.515562,56.279522,2178.699951
2491,2156.000000,2211.949951,2153.050049,2178.699951,11773630.0,42.485238,62.532582,2104.006814,43.548295,2191.100098
2492,2168.500000,2231.899902,2168.000000,2191.100098,9002404.0,41.351413,62.466791,2115.619252,36.150444,2191.050049
2493,2200.000000,2213.800049,2146.600098,2191.050049,6993792.0,47.438579,56.734076,2125.676691,66.145808,2181.949951


1

In [42]:
X

[[492.7789612,
  519.5749512,
  492.3826904,
  509.5202637,
  14236238.0,
  77.73335338,
  63.17854158,
  478.6916457,
  63.02371621]]